In [1311]:
import numpy as np
import heapq

In [1312]:
class Node:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        self.g_score = float('inf')
        self.h_score = 0
        self.f_score = float('inf')
        self.parent = None

    def __repr__(self):
        return f"Node({self.x}, {self.y}, {self.z})"

In [1313]:
def init_nodes(density):
  x_range = (9.5, 11.55)
  y_range = (-10.5, -6)
  z_range = (4.02, 5.57)

  step = (x_range[1] - x_range[0]) / density

  adj = 0.01 #achieve <=

  nodes = []
  for x in np.arange(x_range[0], x_range[1] + adj, step):
      for y in np.arange(y_range[0], y_range[1] + adj, step):
          for z in np.arange(z_range[0], z_range[1] + adj, step):
              node = Node(x, y, z)
              nodes.append(node)

  return nodes, step

def init_zones(safety):
  ko_zones = [[(10.783-safety, -9.8899-safety, 4.8353-safety), (11.071+safety, -9.6929+safety, 5.0665+safety)],
            [(10.8652-safety, -9.0734-safety, 4.3861-safety), (10.9628+safety, -8.7314+safety, 4.6401+safety)],
             [(10.185-safety, -8.3826-safety, 4.1475-safety), (11.665+safety, -8.2826+safety, 4.6725+safety)],
              [(10.7955-safety, -8.0635-safety, 5.1055-safety), (11.3525+safety, -7.7305+safety, 5.1305+safety)],
               [(10.563-safety, -7.1449-safety, 4.6544-safety), (10.709+safety, -6.8099+safety, 4.8164+safety)]]

  ki_zones = [[(10.3, -10.2, 4.32), (11.55, -6, 5.57)], [(9.5, -10.5, 4.02), (10.5, -9.6, 4.8)]]

  return ki_zones, ko_zones


In [1314]:
def inside_zone(point, zone_min_max):
    x = point.x
    y = point.y
    z = point.z
    x_min, y_min, z_min = zone_min_max[0]
    x_max, y_max, z_max = zone_min_max[1]

    if x_min <= x <= x_max and y_min <= y <= y_max and z_min <= z <= z_max:
        return True
    else:
        return False

In [1315]:
def valid_node(Node, ki_zones, ko_zones):
    inside_ki = False
    inside_ko = False

    for zone in ki_zones:
      if inside_zone(Node, zone):
        inside_ki = True

    for zone in ko_zones:
      if inside_zone(Node, zone):
        inside_ko = True

    if inside_ki and not inside_ko:
      return True
    else:
      return False

In [1316]:
def calculate_dist(node1, node2):
  dx = node2.x - node1.x
  dy = node2.y - node1.y
  dz = node2.z - node1.z
  return (dx**2 + dy**2 + dz**2)**0.5

def in_list(cur_node, node_list):
  for node in node_list:
    if cur_node.x == node.x and cur_node.y == node.y and cur_node.z == node.z:
      return node
  return None

def same_node(node1, node2):
  return node1.x == node2.x and node1.y == node2.y and node1.z == node2.z

def get_neighbors(node, nodes, step, start_node, goal_node, ki_zones, ko_zones):
    uneven = False
    if not valid_node(node, ki_zones, ko_zones): return []
    if not in_list(node, nodes): uneven = True

    neighbors = []
    movements = [-1, 0, 1]

    if uneven:
      for curr_node in nodes:
        if (calculate_dist(node, curr_node) < step): neighbors.append(curr_node)
    else:
      if (calculate_dist(node, start_node) < step): neighbors.append(start_node)
      if (calculate_dist(node, goal_node) < step): neighbors.append(goal_node)

      for dx in movements:
        for dy in movements:
            for dz in movements:
                if dx == 0 and dy == 0 and dz == 0:
                    continue

                new_x = node.x + step * dx
                new_y = node.y + step * dy
                new_z = node.z + step * dz

                referenced = in_list(Node(new_x, new_y, new_z), nodes)

                if referenced: neighbors.append(referenced)

    return neighbors

def a_star(start_node, goal_node, nodes, step, ki_zones, ko_zones):
  open_set = []
  closed_set = set()

  start_node.g_score = 0
  start_node.f_score = calculate_dist(start_node, goal_node)
  heapq.heappush(open_set, (start_node.f_score, start_node))

  iter = 0

  while open_set:
    _, current_node = heapq.heappop(open_set)

    if same_node(current_node, goal_node):
      a_path = []
      while current_node:
          a_path.append(current_node)
          current_node = current_node.parent
      a_path.reverse()
      return a_path

    closed_set.add(current_node)

    for neighbor_node in get_neighbors(current_node, nodes, step, start_node, goal_node, ki_zones, ko_zones):
          nodes_closed_set = []

          for c_node in closed_set:
            nodes_closed_set.append(c_node)

          if in_list(neighbor_node, nodes_closed_set):
              continue

          g_score = current_node.g_score + calculate_dist(current_node, neighbor_node)

          if g_score < neighbor_node.g_score:
              neighbor_node.parent = current_node
              neighbor_node.g_score = g_score
              neighbor_node.f_score = g_score + calculate_dist(neighbor_node, goal_node)

              nodes_open_set = []

              for num, o_node in open_set:
                nodes_open_set.append(o_node)

              if not in_list(neighbor_node, nodes_open_set):
                  heapq.heappush(open_set, (neighbor_node.f_score, neighbor_node))

  return None


In [1317]:
def smooth_path(path, ki_zones, ko_zones):
  trimmed_path = [path[0]]

  if (len(path) < 3): return path

  current_index = 0

  while (current_index < len(path)-1):
    max_index = current_index + 1

    for i in range(current_index+2, len(path)):
      line_segment = split_line(path[current_index], path[i], 100)
      if valid_segment(line_segment, ki_zones, ko_zones): max_index = i

    trimmed_path.append(path[max_index])
    current_index = max_index

  return trimmed_path

def valid_segment(segment, ki_zones, ko_zones):
  for point in segment[1:-1]:
    if not valid_node(point, ki_zones, ko_zones):
      return False
  return True

def split_line(start_node, end_node, num_points):
  line_segment = []

  x0, y0, z0 = start_node.x, start_node.y, start_node.z
  x1, y1, z1 = end_node.x, end_node.y, end_node.z

  dx = x1 - x0
  dy = y1 - y0
  dz = z1 - z0

  for i in range(1, num_points + 1):
      t = i / (num_points + 1)
      x = x0 + dx * t
      y = y0 + dy * t
      z = z0 + dz * t
      line_segment.append(Node(x, y, z))

  return line_segment

In [1318]:
def calculatePath(start_input, goal_input):
  start_node = Node(start_input.x, start_input.y, start_input.z)
  goal_node = Node(goal_input.x, goal_input.y, goal_input.z)

  nodes, step = init_nodes(24)
  ki_zones, ko_zones = init_zones(0)

  new_nodes = []

  for node in nodes:
    if valid_node(node, ki_zones, ko_zones): new_nodes.append(node)

  nodes = new_nodes

  unprocessed_path = a_star(start_node, goal_node, nodes, step, ki_zones, ko_zones)
  path = smooth_path(unprocessed_path, ki_zones, ko_zones)

  return path



In [1319]:
def debug_path(path):
  for node in path:
    print(f"this.nodes.push(new Node(new THREE.Vector3({node.x}, {node.z}, {node.y})))")

def setup_path(path, pre):
  for node in path:
    print(pre + f"trajectory.add(new State(new Point({node.x}, {node.y}, {node.z}), new Quaternion(0, 0, -0.707f, 0.707f)));")

In [1320]:
start = Node(9.815, -9.806, 4.293)
goal = Node(11.143, -6.7607, 4.9654)
points = [Node(11.2746, -9.92284, 5.2988), Node(10.612, -9.0709, 4.48), Node(10.71, -7.7, 4.48), Node(10.51, -6.7185, 5.1804), Node(11.114, -7.9756, 5.3393), Node(11.355, -8.9929, 4.7818), Node(11.369, -8.5518, 4.48)]

In [1321]:
'''
for i, point in enumerate(points):
  print("if (goal == {}) {{".format(i+1))
  path = calculatePath(start, point)
  setup_path(path, "    ")
  print("}")
'''

'\nfor i, point in enumerate(points):\n  print("if (goal == {}) {{".format(i+1))\n  path = calculatePath(start, point)\n  setup_path(path, "    ")\n  print("}")\n'

In [1322]:
for i, start_point in enumerate(points):
  print("if (start == {}) {{".format(i+1))
  for j, end_point in enumerate(points):
    if start_point != end_point:
      print("    if (goal == {}) {{".format(j+1))
      path = calculatePath(start_point, end_point)
      setup_path(path, "        ")
      print("    }")

  print("    if (goal == 8) {")
  path = calculatePath(start_point, goal)
  setup_path(path, "        ")
  print("    }")
  print("}")

if (start == 1) {
    if (goal == 2) {
        trajectory.add(new State(new Point(11.2746, -9.92284, 5.2988), new Quaternion(0, 0, -0.707f, 0.707f)));
        trajectory.add(new State(new Point(10.612, -9.0709, 4.48), new Quaternion(0, 0, -0.707f, 0.707f)));
    }
    if (goal == 3) {
        trajectory.add(new State(new Point(11.2746, -9.92284, 5.2988), new Quaternion(0, 0, -0.707f, 0.707f)));
        trajectory.add(new State(new Point(10.71, -7.7, 4.48), new Quaternion(0, 0, -0.707f, 0.707f)));
    }
    if (goal == 4) {
        trajectory.add(new State(new Point(11.2746, -9.92284, 5.2988), new Quaternion(0, 0, -0.707f, 0.707f)));
        trajectory.add(new State(new Point(10.51, -6.7185, 5.1804), new Quaternion(0, 0, -0.707f, 0.707f)));
    }
    if (goal == 5) {
        trajectory.add(new State(new Point(11.2746, -9.92284, 5.2988), new Quaternion(0, 0, -0.707f, 0.707f)));
        trajectory.add(new State(new Point(11.114, -7.9756, 5.3393), new Quaternion(0, 0, -0.707f, 0.707f)));
 